# **[Streamlit Setup] AyikaBot - Generative QA Chatbot for Climate Education**

This project sets up and tests a climate-focused Q&A chatbot using a fine-tuned pre-trained T5 model. It uses Streamlit for the web interface and ngrok for tunneling the local app online from Google Colab. I did this because it is ideal for testing before deploying to Streamlit Cloud.

#### **Features:**
- Loads a custom fine-tuned T5 model for climate education Q&A.
- Runs a Streamlit app to interact with the chatbot.
- Uses ngrok to generate a public link for live testing from Google Colab.

#### **Useful Links:**
- Checkout more about the project here: https://github.com/eadewusic/Domain-Specific-QA-Chatbot-using-Transformer-Models
- You can also find the fine-tuned pretrained model and other files here: https://huggingface.co/Climi/Climate-Education-QA-Chatbot

**Author:** Eunice Adewusi

**Date:** June 2025

In [1]:
!pip install streamlit
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.2 MB/s eta 0:00:00


In [2]:
# Using ngrok tunnel
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦
added 22 packages in 5s
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦

In [3]:
# Import necessary libraries
import streamlit as st
import time
import re
from typing import Tuple, List, Optional
import sys
import os
import subprocess
import threading
import time
import requests
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

In [4]:
# set ngrok authentication token
from pyngrok import ngrok
ngrok.set_auth_token("insert_your_auth_key_from_ngrok_site")

In [ ]:
# Get the correct tunnel password from loca.lt
# this is needed for ngrok to host the app
password = requests.get("https://loca.lt/mytunnelpassword").text.strip()
print(f"\n Tunnel Password: {password}\n")

- When this cell is run, the output displays something like "Tunnel Password: ##.###.##.###" (where # is an actual digit)

- Input that in the custom ngrok page (you'll get a link from the last cell in this notebook) for your app to run

Hiding mine :)

In [6]:
# check folder content
!ls /content/climate_chatbot_BEST_exp4c

added_tokens.json	       generation_config.json	spiece.model
ARCHITECTURE.md		       model_architecture.json	tf_model.h5
ayikabot_complete_pipeline.py  optimal_generation.py	tokenizer_config.json
comprehensive_results.json     run_chatbot.py
config.json		       special_tokens_map.json


In [7]:
# Load the best fine-tuned pre-trained T5 model and tokenizer
model = TFT5ForConditionalGeneration.from_pretrained("/content/climate_chatbot_BEST_exp4c")
tokenizer = T5Tokenizer.from_pretrained("/content/climate_chatbot_BEST_exp4c")

# Save loaded model and tokenizer to a new directory
model.save_pretrained("/content/ayikabot_clean")
tokenizer.save_pretrained("/content/ayikabot_clean")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/climate_chatbot_BEST_exp4c.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


('/content/ayikabot_clean/tokenizer_config.json',
 '/content/ayikabot_clean/special_tokens_map.json',
 '/content/ayikabot_clean/spiece.model',
 '/content/ayikabot_clean/added_tokens.json')

You can also find the fine-tuned pretrained model and other files here: https://huggingface.co/Climi/Climate-Education-QA-Chatbot

This was done to get a more focused and clean directory with the very important files for deployment

In [8]:
# The "/content/ayikabot_streamlit_app.py" file contains all the code needed for streamlit to run.
# Whatever is to be changed on the UI should be changed in this file
def run_streamlit():
    with open("streamlit_log.txt", "w") as f:
        subprocess.run(
            ["streamlit", "run", "/content/ayikabot_streamlit_app.py", "--server.port", "8501", "--server.headless", "true"],
            stdout=f,
            stderr=subprocess.STDOUT
        )

# Start Streamlit in a background thread
streamlit_thread = threading.Thread(target=run_streamlit)
streamlit_thread.start()

# Wait for Streamlit to boot up fully
print("Waiting for Streamlit to start...")
time.sleep(20)  # Increased wait time to 20 seconds for larger applications

# Kill any existing ngrok tunnels before starting a new one
print("Killing any existing ngrok tunnels...")
ngrok.kill()
time.sleep(5) # Give ngrok time to shut down

# Now start ngrok tunnel AFTER Streamlit is fully running
try:
    print("Starting new ngrok tunnel...")
    public_url = ngrok.connect(8501).public_url # Access the public_url attribute
    print(f"App is live at: {public_url}")
except Exception as e:
    print(f"Failed to start ngrok tunnel: {e}")

Waiting for Streamlit to start...
Killing any existing ngrok tunnels...
Starting new ngrok tunnel...
App is live at: https://02a0-34-125-57-200.ngrok-free.app


The streamlit app link is https://02a0-34-125-57-200.ngrok-free.app

This still needs to be deployed because the generated public URL becomes invalid when Colab runtime stops, as the ngrok tunnel closes. I used this approach to test the code and functionalities before deploying to Streamlit Cloud